# Import

In [ ]:
import pandas as pd
import numpy as np
import requests

# Test de l'API

In [114]:
# Définir l'URL de l'API et le lien du fichier à charger
file_path = "fichiers tests/application_test_2_clients.csv"
url = "http://localhost:8000/predict"

# Ouvrir le fichier et l'envoyer à l'API
with open(file_path, 'rb') as f:
    files = {'file': (file_path, f, 'text/csv')}
    response = requests.post(url, files=files)

# Récupérer la réponse
results = response.json()
    
# Convertir le résultat en en DataFrame
results_df = pd.DataFrame(results)

# Afficher le résultat
results_df

,SK_ID_CURR,PROBA_DEFAUT,PRED_DEFAUT,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,0.348303,0,0.038764,-0.056866,0.016126,0.002913,-0.003927,-0.006212,0.107460,...,0.004039,0.0,0.0,0.0,0.000017,-0.000090,0.001761,0.003911,0.006501,-0.062504
1,100005,0.676774,1,0.022155,0.266427,0.084031,0.002745,0.001637,0.000941,-0.100068,...,0.002432,0.0,0.0,0.0,0.000026,-0.000241,0.000971,0.008564,0.015005,0.008866


# Créer les fichiers tests

In [ ]:
# Lire le fichier à partir à modifier pour créer les fichiers tests
df = pd.read_csv("fichiers tests/application_test_2_clients.csv", index_col=0)

# 1. Créer un fichier ne contenant que l'entête
empty_rows = df.drop(df.index)
empty_rows.to_csv("fichiers tests/application_test_empty_rows.csv")

# 2. Créer un fichier contenant une ligne totalement vide
one_empty_row = pd.concat([df] * 2, ignore_index=True)
one_empty_row.iloc[2] = np.nan
one_empty_row.to_csv("fichiers tests/application_test_one_empty_row.csv")

# 3. Créer un fichier avec des colonnes manquantes
missing_columns = df[df.columns[:50]]
missing_columns.to_csv("fichiers tests/application_test_missing_columns.csv")

# 4. Créer un fichier xlsx
df.to_excel("fichiers tests/application_test_2_clients.xlsx")

# 5. Créer un fichier beaucoup trop lourd
heavy_file = pd.concat([df] * 50000, ignore_index=True)
heavy_file.to_csv("fichiers tests/application_test_heavy_file.csv")

# 6. Créer un fichier vide
empty_file = pd.DataFrame()
empty_file.to_csv("fichiers tests/application_test_empty_file.csv")

# 7. Créer un fichier avec nombre au format texte
wrong_dtype = df.copy()
wrong_dtype['AMT_INCOME_TOTAL'] = wrong_dtype['AMT_INCOME_TOTAL'].astype(str)
wrong_dtype.to_csv("fichiers tests/application_test_wrong_dtype.csv")

# 8. Créer fichier avec du texte dans une colonne qui devrait ne contenir que des nombres
wrong_values = df.copy()
wrong_values['AMT_INCOME_TOTAL'] = 'Donnée incohérente'
wrong_values.to_csv("fichiers tests/application_test_wrong_values.csv")

# Script tests

In [ ]:
import pytest
import requests
import logging

# Configuration du logging pour pytest
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

########################################## 1. Test fichier ne contenant que l'entête ##########################################

def test_fichier_entete_seul():
    """Test fichier ne contenant que l'entête"""
    # Définir l'url, le nom du test et le lien du fichier à utiliser
    url = "http://localhost:8000/predict"
    test_name = "Test fichier ne contenant que l'entête"
    file_path = "fichiers tests/application_test_empty_rows.csv"
    
    # Ouvrir le fichier et l'envoyer à l'API et récupérer la réponse
    with open(file_path, 'rb') as f:
        files = {'file': (file_path, f, 'text/csv')}
        response = requests.post(url, files=files)
    
    # Enregistrer les détails du test
    logger.info(test_name)
    logger.info(f"Status code: {response.status_code}")
    logger.info(f"Status détail: {response.json()['detail']}")

    # Assert pour validation pytest
    assert response.status_code == 400, f"Attendu 400, reçu {response.status_code}"

##################################### 2. Test fichier contenant une ligne totalement vide #####################################

def test_fichier_une_ligne_vide():
    """Test fichier une ligne totalement vide"""
    # Définir l'url, le nom du test et le lien du fichier à utiliser
    url = "http://localhost:8000/predict"
    test_name = "Test fichier une ligne totalement vide"
    file_path = "fichiers tests/application_test_one_empty_row.csv"
    
    # Ouvrir le fichier et l'envoyer à l'API et récupérer la réponse
    with open(file_path, 'rb') as f:
        files = {'file': (file_path, f, 'text/csv')}
        response = requests.post(url, files=files)
    
    # Enregistrer les détails du test
    logger.info(test_name)
    logger.info(f"Status code: {response.status_code}")
    logger.info(f"Status détail: {response.json()['detail']}")

    # Assert pour validation pytest
    assert response.status_code == 400, f"Attendu 400, reçu {response.status_code}"

######################################## 3. Test fichier avec des colonnes manquantes #########################################

def test_fichier_colonnes_manquantes():
    """Test fichier avec des colonnes manquantes"""
    # Définir l'url, le nom du test et le lien du fichier à utiliser
    url = "http://localhost:8000/predict"
    test_name = "Test fichier avec des colonnes manquantes"
    file_path = "fichiers tests/application_test_missing_columns.csv"
    
    # Ouvrir le fichier et l'envoyer à l'API et récupérer la réponse
    with open(file_path, 'rb') as f:
        files = {'file': (file_path, f, 'text/csv')}
        response = requests.post(url, files=files)
    
    # Enregistrer les détails du test
    logger.info(test_name)
    logger.info(f"Status code: {response.status_code}")
    logger.info(f"Status détail: {response.json()['detail']}")

    # Assert pour validation pytest
    assert response.status_code == 400, f"Attendu 400, reçu {response.status_code}"

#################################################### 4. Test fichier xlsx #####################################################

def test_fichier_xlsx():
    """Test fichier xlsx"""
    # Définir l'url, le nom du test et le lien du fichier à utiliser
    url = "http://localhost:8000/predict"
    test_name = "Test fichier xlsx"
    file_path = "fichiers tests/application_test_2_clients.xlsx"
    
    # Ouvrir le fichier et l'envoyer à l'API et récupérer la réponse
    with open(file_path, 'rb') as f:
        files = {'file': (file_path, f, 'text/csv')}
        response = requests.post(url, files=files)
    
    # Enregistrer les détails du test
    logger.info(test_name)
    logger.info(f"Status code: {response.status_code}")
    logger.info(f"Status détail: {response.json()['detail']}")

    # Assert pour validation pytest
    assert response.status_code == 422, f"Attendu 422, reçu {response.status_code}"

############################################ 5. Test fichier beaucoup trop lourd ##############################################

def test_fichier_trop_lourd():
    """Test fichier beaucoup trop lourd"""
    # Définir l'url, le nom du test et le lien du fichier à utiliser
    url = "http://localhost:8000/predict"
    test_name = "Test fichier beaucoup trop lourd"
    file_path = "fichiers tests/application_test_heavy_file.csv"
    
    # Ouvrir le fichier et l'envoyer à l'API et récupérer la réponse
    with open(file_path, 'rb') as f:
        files = {'file': (file_path, f, 'text/csv')}
        response = requests.post(url, files=files)
    
    # Enregistrer les détails du test
    logger.info(test_name)
    logger.info(f"Status code: {response.status_code}")
    logger.info(f"Status détail: {response.json()['detail']}")

    # Assert pour validation pytest
    assert response.status_code == 413, f"Attendu 413, reçu {response.status_code}"

#################################################### 6. Test fichier vide #####################################################

def test_fichier_vide():
    """Test fichier vide"""
    # Définir l'url, le nom du test et le lien du fichier à utiliser
    url = "http://localhost:8000/predict"
    test_name = "Test fichier vide"
    file_path = "fichiers tests/application_test_empty_file.csv"
    
    # Ouvrir le fichier et l'envoyer à l'API et récupérer la réponse
    with open(file_path, 'rb') as f:
        files = {'file': (file_path, f, 'text/csv')}
        response = requests.post(url, files=files)
    
    # Enregistrer les détails du test
    logger.info(test_name)
    logger.info(f"Status code: {response.status_code}")
    logger.info(f"Status détail: {response.json()['detail']}")

    # Assert pour validation pytest
    assert response.status_code == 400, f"Attendu 400, reçu {response.status_code}"

######################################## 7. Test fichier avec nombre au format texte ##########################################

def test_fichier_format_erroné():
    """Test fichier avec nombre au format texte"""
    # Définir l'url, le nom du test et le lien du fichier à utiliser
    url = "http://localhost:8000/predict"
    test_name = "Test fichier avec nombre au format texte"
    file_path = "fichiers tests/application_test_wrong_dtype.csv"
    
    # Ouvrir le fichier et l'envoyer à l'API et récupérer la réponse
    with open(file_path, 'rb') as f:
        files = {'file': (file_path, f, 'text/csv')}
        response = requests.post(url, files=files)
    
    # Enregistrer les détails du test
    logger.info(test_name)
    logger.info(f"Status code: {response.status_code}")

    # Assert pour validation pytest
    assert response.status_code == 200, f"Attendu 200, reçu {response.status_code}"

#################### 8. Test fichier avec du texte dans une colonne qui devrait ne contenir que des nombres ###################

def test_fichier_valeur_incohérente():
    """Test fichier avec du texte dans une colonne qui devrait ne contenir que des nombres"""
    # Définir l'url, le nom du test et le lien du fichier à utiliser
    url = "http://localhost:8000/predict"
    test_name = "Test fichier avec du texte dans une colonne qui devrait ne contenir que des nombres"
    file_path = "fichiers tests/application_test_wrong_values.csv"
    
    # Ouvrir le fichier et l'envoyer à l'API et récupérer la réponse
    with open(file_path, 'rb') as f:
        files = {'file': (file_path, f, 'text/csv')}
        response = requests.post(url, files=files)
    
    # Enregistrer les détails du test
    logger.info(test_name)
    logger.info(f"Status code: {response.status_code}")
    logger.info(f"Status détail: {response.json()['detail']}")

    # Assert pour validation pytest
    assert response.status_code == 400, f"Attendu 400, reçu {response.status_code}"

##################################################### 9. Test sans fichier ####################################################

def test_sans_fichier():
    """Test sans fichier"""
    # Définir l'url
    url = "http://localhost:8000/predict"
    test_name = "Test sans fichier"
    
    # Envoyer une requête à l'API sans fichier
    response = requests.post(url)
    
    # Enregistrer les détails du test
    logger.info(test_name)
    logger.info(f"Status code: {response.status_code}")
    logger.info(f"Status détail: {response.json()['detail']}")

    # Assert pour validation pytest
    assert response.status_code == 422, f"Attendu 422, reçu {response.status_code}"